In [15]:
import pandas as pd

In [16]:
dataset = pd.read_csv('data/processed/openhust_with_id.csv')

In [17]:
dataset.head()

,ID,Document,Summary,Dataset
0,1,"Theo đó , các sở , ngành trên địa bàn phải vào...","Các quận , huyện , thị xã tuyên truyền bằng nh...",vietnews
1,2,"Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ...",Cháu bé được phát hiện trong tình trạng chưa c...,vietnews
2,3,"Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ...",Báo tuyết ( Panthera uncia ) được mệnh danh là...,vietnews
3,4,"Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ...","Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...",vietnews
4,5,Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...,"Loại "" huyết thanh nói thật "" mà CIA sử dụng đ...",vietnews


In [18]:
dataset.rename(columns={'Document': 'Text'}, inplace=True)

In [19]:
dataset = dataset[['Summary','Text']]
dataset = dataset.head(10000)

In [20]:
dataset.head()

,Summary,Text
0,"Các quận , huyện , thị xã tuyên truyền bằng nh...","Theo đó , các sở , ngành trên địa bàn phải vào..."
1,Cháu bé được phát hiện trong tình trạng chưa c...,"Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng ..."
2,Báo tuyết ( Panthera uncia ) được mệnh danh là...,"Thoạt đầu nhìn vào bức ảnh , nếu không có dấu ..."
3,"Thấy con chim bị mắc bẫy rơi xuống hồ nước , a...","Khoảng 9h30 sáng 5/6 , anh Đào Nhật Tuấn ( 30 ..."
4,"Loại "" huyết thanh nói thật "" mà CIA sử dụng đ...",Liên minh Tự do Dân sự Mỹ ( ACLU ) đã nhận đượ...


In [21]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [22]:
full_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['Summary', 'Text'],
    num_rows: 8000
})
Dataset({
    features: ['Summary', 'Text'],
    num_rows: 2000
})


In [25]:
import torch
import pprint
import evaluate
import numpy as np

In [26]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

In [27]:
MODEL = 't5-small'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 5
OUT_DIR = './result666'
MAX_LENGTH = 512 

In [28]:
tokenizer = T5Tokenizer('models/trained_spiece.model')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [29]:
from functools import partial
from transformers import T5Tokenizer

# Initialize the tokenizer
# tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer = T5Tokenizer('models/trained_spiece.model')
# Function to convert text data into model inputs and targets
def preprocess_function(examples, tokenizer):
    inputs = [f"summarize: {text}" for text in examples['Text']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create a partial function with the tokenizer
preprocess_function_with_tokenizer = partial(preprocess_function, tokenizer=tokenizer)

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)


Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [31]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [32]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [33]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [35]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS, # EPOCHS = 4
    per_device_train_batch_size=BATCH_SIZE, # BATCH_SIZE = 4
    per_device_eval_batch_size=BATCH_SIZE, 
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=400,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]

{'loss': 0.5505, 'grad_norm': 0.6501509547233582, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.55, 'grad_norm': 0.5168552398681641, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.01}
{'loss': 0.548, 'grad_norm': 0.7078872323036194, 'learning_rate': 6e-06, 'epoch': 0.01}
{'loss': 0.5454, 'grad_norm': 0.5406089425086975, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.5765, 'grad_norm': 0.7442662119865417, 'learning_rate': 1e-05, 'epoch': 0.03}
{'loss': 0.6236, 'grad_norm': 0.6773218512535095, 'learning_rate': 1.2e-05, 'epoch': 0.03}
{'loss': 0.511, 'grad_norm': 0.5719890594482422, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.04}
{'loss': 0.5632, 'grad_norm': 0.8859125375747681, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.04}
{'loss': 0.6622, 'grad_norm': 0.5398176312446594, 'learning_rate': 1.8e-05, 'epoch': 0.04}
{'loss': 0.5168, 'grad_norm': 0.4715711772441864, 'learning_rate': 2e-05, 'epoch': 0.05}
{'loss': 0.5524, 'grad_norm

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.47695666551589966, 'eval_rouge1': 0.6112, 'eval_rouge2': 0.31, 'eval_rougeL': 0.4628, 'eval_gen_len': 42.2295, 'eval_runtime': 93.3387, 'eval_samples_per_second': 21.427, 'eval_steps_per_second': 5.357, 'epoch': 0.2}
{'loss': 0.5204, 'grad_norm': 0.6771959066390991, 'learning_rate': 8.2e-05, 'epoch': 0.2}
{'loss': 0.5303, 'grad_norm': 0.4466670751571655, 'learning_rate': 8.4e-05, 'epoch': 0.21}
{'loss': 0.4724, 'grad_norm': 0.5213791131973267, 'learning_rate': 8.6e-05, 'epoch': 0.21}
{'loss': 0.5627, 'grad_norm': 0.605518102645874, 'learning_rate': 8.800000000000001e-05, 'epoch': 0.22}
{'loss': 0.4768, 'grad_norm': 0.3355215787887573, 'learning_rate': 9e-05, 'epoch': 0.23}
{'loss': 0.4898, 'grad_norm': 0.30185094475746155, 'learning_rate': 9.200000000000001e-05, 'epoch': 0.23}
{'loss': 0.5331, 'grad_norm': 0.6162463426589966, 'learning_rate': 9.4e-05, 'epoch': 0.23}
{'loss': 0.473, 'grad_norm': 1.788686752319336, 'learning_rate': 9.6e-05, 'epoch': 0.24}
{'loss': 0.4675,

  0%|          | 0/500 [00:00<?, ?it/s]

...

{'eval_loss': 0.37647417187690735, 'eval_rouge1': 0.6653, 'eval_rouge2': 0.3555, 'eval_rougeL': 0.5057, 'eval_gen_len': 43.1525, 'eval_runtime': 97.7539, 'eval_samples_per_second': 20.46, 'eval_steps_per_second': 5.115, 'epoch': 4.6}
{'loss': 0.3786, 'grad_norm': 0.4645114839076996, 'learning_rate': 8.315789473684212e-06, 'epoch': 4.61}
{'loss': 0.3215, 'grad_norm': 0.28273457288742065, 'learning_rate': 8.210526315789475e-06, 'epoch': 4.61}
{'loss': 0.3744, 'grad_norm': 0.46975570917129517, 'learning_rate': 8.105263157894736e-06, 'epoch': 4.62}
{'loss': 0.3889, 'grad_norm': 0.30785834789276123, 'learning_rate': 8.000000000000001e-06, 'epoch': 4.62}
{'loss': 0.342, 'grad_norm': 0.36643776297569275, 'learning_rate': 7.894736842105263e-06, 'epoch': 4.62}
{'loss': 0.342, 'grad_norm': 0.5422959923744202, 'learning_rate': 7.789473684210528e-06, 'epoch': 4.63}
{'loss': 0.3125, 'grad_norm': 0.29334431886672974, 'learning_rate': 7.68421052631579e-06, 'epoch': 4.63}
{'loss': 0.3753, 'grad_norm':

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.3764376938343048, 'eval_rouge1': 0.6651, 'eval_rouge2': 0.3551, 'eval_rougeL': 0.5051, 'eval_gen_len': 43.1525, 'eval_runtime': 91.9166, 'eval_samples_per_second': 21.759, 'eval_steps_per_second': 5.44, 'epoch': 4.8}
{'loss': 0.3772, 'grad_norm': 0.5875990390777588, 'learning_rate': 4.105263157894737e-06, 'epoch': 4.8}
{'loss': 0.3969, 'grad_norm': 0.4689684808254242, 'learning_rate': 4.000000000000001e-06, 'epoch': 4.81}
{'loss': 0.3748, 'grad_norm': 0.34493255615234375, 'learning_rate': 3.894736842105264e-06, 'epoch': 4.81}
{'loss': 0.3576, 'grad_norm': 0.3136988878250122, 'learning_rate': 3.789473684210527e-06, 'epoch': 4.82}
{'loss': 0.4399, 'grad_norm': 0.3877088725566864, 'learning_rate': 3.6842105263157892e-06, 'epoch': 4.83}
{'loss': 0.3627, 'grad_norm': 0.35851752758026123, 'learning_rate': 3.5789473684210525e-06, 'epoch': 4.83}
{'loss': 0.3783, 'grad_norm': 0.3726615905761719, 'learning_rate': 3.4736842105263158e-06, 'epoch': 4.83}
{'loss': 0.3339, 'grad_norm'

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 0.3762052655220032, 'eval_rouge1': 0.6654, 'eval_rouge2': 0.3551, 'eval_rougeL': 0.5051, 'eval_gen_len': 43.1525, 'eval_runtime': 93.546, 'eval_samples_per_second': 21.38, 'eval_steps_per_second': 5.345, 'epoch': 5.0}
{'train_runtime': 4451.3826, 'train_samples_per_second': 8.986, 'train_steps_per_second': 2.246, 'train_loss': 0.40834275386333463, 'epoch': 5.0}


In [36]:
model_path = f"{OUT_DIR}/checkpoint-10000"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer('models/trained_spiece.model')

In [37]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=200,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [38]:
text = 'Khu vực Nam Bộ có nắng nóng và nắng nóng gay gắt trong các ngày từ 1-8/5; sau đó từ 9/5, nắng nóng chủ yếu tập trung tại các tỉnh miền Đông Nam Bộ. Trên cả nước đã ghi nhận nhiều trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày vượt giá trị lịch sử. Cụ thể, cùng trong ngày 1/5, ở Đông Hà (Quảng Trị) đạt đến 43,2 độ, vượt mức 42,3 độ năm 2023; Huế (Thừa Thiên Huế) 42,1 độ, vượt kỷ lục 41,3 độ năm 1983; Đà Nẵng 41,5, vượt mức 40,5 năm 1983 hay Thủ Dầu Một (Bình Dương) 38,9 độ ngày 2/5, vượt mức 38,7 năm 2016,…'
print(text)

Khu vực Nam Bộ có nắng nóng và nắng nóng gay gắt trong các ngày từ 1-8/5; sau đó từ 9/5, nắng nóng chủ yếu tập trung tại các tỉnh miền Đông Nam Bộ. Trên cả nước đã ghi nhận nhiều trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày vượt giá trị lịch sử. Cụ thể, cùng trong ngày 1/5, ở Đông Hà (Quảng Trị) đạt đến 43,2 độ, vượt mức 42,3 độ năm 2023; Huế (Thừa Thiên Huế) 42,1 độ, vượt kỷ lục 41,3 độ năm 1983; Đà Nẵng 41,5, vượt mức 40,5 năm 1983 hay Thủ Dầu Một (Bình Dương) 38,9 độ ngày 2/5, vượt mức 38,7 năm 2016,…


In [40]:
summarize_text(text, model, tokenizer, max_length=1000)

'Trong nắng nóng chủ yếu tập trung tại các tỉnh miên Đông Nam Bộ. Trên cả nước đã ghi nhận nhiêu trạm khí tượng xảy ra giá trị nhiệt độ cao nhất ngày vượt giá trị lịch sử. Cụ thể, cùng trong ngày 1/5, ở Đông Hà (Quảng Trị) đạt đến 43,2 độ, vượt mức 38,7 năm 1983; Đà Nẵng 41,5.'

In [58]:
import json

# Đọc file JSON
with open('result666/checkpoint-10000/trainer_state.json', 'r') as file:
    data = json.load(file)

# Lấy từ điển có trường 'log_history'
log_history = data.get('log_history', [])


filtered_logs = [log for log in log_history if log.get('step', 0) % 400 == 0 and 'eval_rouge1' in log]

# In ra các từ điển đã lọc được
for log in filtered_logs:
    print(log)



{'epoch': 0.2, 'eval_gen_len': 42.2295, 'eval_loss': 0.47695666551589966, 'eval_rouge1': 0.6112, 'eval_rouge2': 0.31, 'eval_rougeL': 0.4628, 'eval_runtime': 93.3387, 'eval_samples_per_second': 21.427, 'eval_steps_per_second': 5.357, 'step': 400}
{'epoch': 0.4, 'eval_gen_len': 43.0795, 'eval_loss': 0.4514653980731964, 'eval_rouge1': 0.6171, 'eval_rouge2': 0.3189, 'eval_rougeL': 0.4718, 'eval_runtime': 91.8426, 'eval_samples_per_second': 21.776, 'eval_steps_per_second': 5.444, 'step': 800}
{'epoch': 0.6, 'eval_gen_len': 43.1525, 'eval_loss': 0.43480512499809265, 'eval_rouge1': 0.6147, 'eval_rouge2': 0.3189, 'eval_rougeL': 0.472, 'eval_runtime': 92.6325, 'eval_samples_per_second': 21.591, 'eval_steps_per_second': 5.398, 'step': 1200}
{'epoch': 0.8, 'eval_gen_len': 43.152, 'eval_loss': 0.42746350169181824, 'eval_rouge1': 0.6266, 'eval_rouge2': 0.3268, 'eval_rougeL': 0.4793, 'eval_runtime': 93.2788, 'eval_samples_per_second': 21.441, 'eval_steps_per_second': 5.36, 'step': 1600}
{'epoch': 1.

In [63]:
import pandas as pd

# Chuyển danh sách từ điển thành DataFrame
df = pd.DataFrame(filtered_logs)

# Tạo một bản sao của DataFrame
styled_df = df.style

# Tùy chỉnh màu sắc cho cột 'eval_rouge1'
styled_df = styled_df.apply(lambda row: ['color: blue' if val is None else '' for val in row], axis=1, subset=['eval_rouge1'])

# Hiển thị DataFrame đã tùy chỉnh với màu sắc
styled_df


,epoch,eval_gen_len,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_runtime,eval_samples_per_second,eval_steps_per_second,step
0,0.200000,42.229500,0.476957,0.611200,0.310000,0.462800,93.338700,21.427000,5.357000,400
1,0.400000,43.079500,0.451465,0.617100,0.318900,0.471800,91.842600,21.776000,5.444000,800
2,0.600000,43.152500,0.434805,0.614700,0.318900,0.472000,92.632500,21.591000,5.398000,1200
3,0.800000,43.152000,0.427464,0.626600,0.326800,0.479300,93.278800,21.441000,5.360000,1600
4,1.000000,43.152500,0.421790,0.633900,0.329700,0.480800,97.281900,20.559000,5.140000,2000
5,1.200000,43.152500,0.414433,0.629700,0.329400,0.481600,96.611600,20.701000,5.175000,2400
6,1.400000,43.152500,0.410623,0.642000,0.336300,0.486300,96.871500,20.646000,5.161000,2800
7,1.600000,43.152500,0.404621,0.641000,0.336900,0.487900,98.656800,20.272000,5.068000,3200
8,1.800000,43.152500,0.401180,0.648800,0.340500,0.491100,95.524400,20.937000,5.234000,3600
9,2.000000,43.152500,0.396773,0.650200,0.341600,0.492900,94.007600,21.275000,5.319000,4000


In [64]:
import gradio as gr
 
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [65]:
def summarize_text(text):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=512,
        truncation=True,
        padding='max_length'
    )
 
    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=50,
        num_beams=5,
        # early_stopping=True
    )
 
    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [66]:
model_path = "result666/checkpoint-10000" 
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer('models/trained_spiece.model')

In [67]:
interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=10, placeholder='Enter Text Here...', label='Input text'),
    outputs=gr.Textbox(label='Summarized Text'),
    title='Text Summarizer using T5'
)
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
